# Multiple Inheritance

In [1]:
class BaseClass(object):
    
    def __init__(self, value):
        self.value = value
        
class TimesTwo(object):

    def __init__(self):
        self.value *= 2

class AddTwo(object):
    
    def __init__(self):
        self.value += 2

Classes with multiple inheritance can lead to some very bizarre  behaviour. Above I have defined some classes: `BaseClass`, `TimesTwo` and an `AddTwo` class. Below, I create another class which inherits from all of the above classes.

In [2]:
class InheritAll(BaseClass, TimesTwo, AddTwo):
    
    def __init__(self, value):
        BaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        AddTwo.__init__(self)

Here the ordering of the parent classes are the same as in the constructor. Now if we call this class we should get a value of 12

In [3]:
foo = InheritAll(5)
foo.value

12

Awesome, we get a value of 12 as predicted. Now let's re-define the `InheritAll` class by changing the order of parent class, but keeping the order in the constructor the same:

In [4]:
class InheritAll(BaseClass, AddTwo, TimesTwo):
    
    def __init__(self, value):
        BaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        AddTwo.__init__(self)

In [5]:
foo = InheritAll(5)
foo.value

12

We still get 12... this can cause some confusion, if the orders do not match. Another problem can occur with diamond inheritance where a class inherits two different parent classes, and those two parent classes inherit a common class somewhere in the heirarchy. Diamond inheritance will cause the `__init__` method of common super classes to run multiple times. Let's re-define `TimesTwo` and `AddTwo` to inherit from `BaseClass`:

In [6]:
class TimesTwo(BaseClass):

    def __init__(self, value):
        BaseClass.__init__(self, value)
        self.value *= 2

class AddTwo(BaseClass):
    
    def __init__(self, value):
        BaseClass.__init__(self, value)
        self.value += 2

I now define a `Diamond` class which inherits from both of these classes, therefore, `BaseClass` will be at the top of the diamond.

In [7]:
class Diamond(TimesTwo, AddTwo):
    
    def __init__(self, value):
        TimesTwo.__init__(self, value)
        AddTwo.__init__(self, value)

In [8]:
foo = Diamond(5)
foo.value

7

The output should be 12, what is going on here? The reason `foo.value` is 7 and not 12 is because the second parent's class constructor (`AddTwo.__init__`) actually resets the value back to 5 when `BaseClass.__init__` gets called the second time.

To get around this python has a method resolution order (MRO). The MRO standardises the order in which super classes are initialised and ensure common super classes are only run once. I now re-create the diamond hierarchy, but this time using `super` to initialise the parent class:

In [9]:
class TimesTwo(BaseClass):

    def __init__(self, value):
        super(TimesTwo, self).__init__(value)
        self.value *= 2

class AddTwo(BaseClass):
    
    def __init__(self, value):
        super(AddTwo, self).__init__(value)
        self.value += 2

In [10]:
class Diamond(TimesTwo, AddTwo):
    
    def __init__(self, value):
        super(Diamond, self).__init__(value)

In [11]:
foo = Diamond(5)
foo.value

14

Hang on, shouldn't the answer be 12 (5 * 2 + 2)? Becuase `TimesTwo.__init__` is called first? Well no, even though the MRO matches the order defined in the class. To get a better understanding of what's happening let's have a look at the MRO for the `Diamond` class:

In [12]:
from pprint import pprint
pprint(Diamond.mro())

[<class '__main__.Diamond'>,
 <class '__main__.TimesTwo'>,
 <class '__main__.AddTwo'>,
 <class '__main__.BaseClass'>,
 <class 'object'>]


When the `Diamond` class is called it in turn calls `TimesTwo.__init__` and then `AddTwo.__init__` followed by `BaseClass.__init__`. Once the top of the diamond is reached, the initialisations happen is reverse order to what is present in the MRO. So, `BaseClass.__init__` assigns value to 5, then  `AddTwo.__init__` adds two (value is now 7). Finally, `TimesTwo.__init__` is called and the value is now 14.

The `super` built in function works well but the syntax is very verbose. The class you're in has to be specified along with the method name and all the arguments. If the class name is ever changed then the all the calls to `super` will also have to be updated.

Luckily, python 3 fixes these issues by making a call to `super` with no arguments: 

In [13]:
class TimesTwo(BaseClass):

    def __init__(self, value):
        super(__class__, self).__init__(value * 2)
        
# This also works
class AddTwo(BaseClass):
    
    def __init__(self, value):
        super().__init__(value + 2)

In [14]:
TimesTwo(5).value

10

In [15]:
AddTwo(5).value

7